- OpenClassrooms Project 7, Data Scientist
- Author : Oumeima EL GHARBI
- Date : November, December 2022

### Importing libraries

In [39]:
%reset -f

import warnings
warnings.filterwarnings(action="ignore")

# to compute time of pipeline
from time import time, strftime, gmtime

import pandas as pd
from os import listdir
from os.path import isfile, join

import joblib

%matplotlib inline
%autosave 300

Autosaving every 300 seconds


In [40]:
import matplotlib.pyplot as plt
import seaborn as sns
from evaluation_metrics import *

In [41]:
import sys
print("Python version {}.{}.{}".format(str(sys.version_info[0]), str(sys.version_info[1]), str(sys.version_info[2])))

Python version 3.9.7


In [42]:
# Starting time
t0 = time()

input_path = "./dataset/source/"
output_path = "./dataset/cleaned/"

## Part I : Exploratory Data Analysis

#### Importing all datasets

We read the folder in which we have all the csv files.

In [43]:
all_files = [f for f in listdir(input_path) if isfile(join(input_path, f))]

all_csv_files = [file for file in all_files if file.endswith(".csv")]
print(all_files, end='\n\n')

print(all_csv_files)

['application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv']

['application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv']


In [44]:
def load_data(path, filename):
    """
    Step 0)
    :param path:
    :param filename: (string)
    :return:
    """
    print("___Loading raw dataset___")

    # Load raw data
    dataset_file = "{}{}".format(path, filename)
    dataset = pd.read_csv(dataset_file, encoding="windows-1252")

    print("Initial shape :", dataset.shape)
    return dataset

In [45]:
all_datasets = []

for index, dataset_csv in enumerate(all_csv_files):
    name = dataset_csv.split(".")[0]
    #if name.startswith("olist"):
     #   list_name = name.split("_")[1:-1]
      #  name = "_".join(list_name)
    # we make the dataset as a global variable so we can use it later in the notebook
    globals()["dataset_" + name] = load_data(input_path, dataset_csv)

    # we add the dataset to the list of all datasets.
    #all_datasets.append(globals()["dataset_" + name])

    print("This is the dataset n°{} : {}".format(index + 1, name))
    df = globals()["dataset_" + name]
    all_datasets.append(("dataset_" + name, df))
    display(df.head(2))
    display(df.info(verbose=True))

___Loading raw dataset___


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\oumei\AppData\Local\Temp/ipykernel_2508/2910128673.py", line 9, in <module>
    globals()["dataset_" + name] = load_data(input_path, dataset_csv)
  File "C:\Users\oumei\AppData\Local\Temp/ipykernel_2508/531502676.py", line 12, in load_data
    dataset = pd.read_csv(dataset_file, encoding="windows-1252")
  File "C:\Users\oumei\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\oumei\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\oumei\AppData\Roaming\Python\Python39\site-packages\pandas\io\parsers\readers.py", line 950, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "C:\

TypeError: object of type 'NoneType' has no len()

For each dataset, we display its dimension and if it has missing values (NaN)

In [ ]:
print("Shape dataset")
for tuple_df in all_datasets :
    print("The dataset called :", tuple_df[0], "has a shape :", tuple_df[1].shape)

In [ ]:
print("Missing values")
for tuple_df in all_datasets :
    df = tuple_df[1]
    print("The dataset called :", tuple_df[0], "has :", round(df.isna().sum().sum() / (df.shape[0] * df.shape[1]) * 100), "% of missing values.")

In [ ]:
dataset_application_train.isna().sum(axis=0)

In [ ]:
print("Duplicated rows ?")
for tuple_df in all_datasets:
    print("The dataset called :", tuple_df[0], "has :",tuple_df[1].duplicated().sum(), "duplicated rows.")

##### Distribution of the target

In [ ]:
dataset_application_train.TARGET.value_counts(normalize=True).plot(kind="pie")

In [ ]:
dataset_application_train.TARGET.value_counts(normalize=True)

In [ ]:
print("SMOTE to do")

## Part II : Feature Engineering

##### Kernel for the feature engineering (Kaggle : https://www.kaggle.com/code/jsaguiar/lightgbm-with-simple-features/script)

In [ ]:
from feature_engineering import *
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [ ]:
data = pd.read_csv("dataset/cleaned/df_cleaned.csv")

In [ ]:
data[data.TARGET.isna()]

In [ ]:
data.isna().sum(axis=0)

In [ ]:
print(data.shape)
# exclude name column
X = data.loc[:, data.columns != 'TARGET']
print(X.shape)

y = data.TARGET
print(y.shape)

In [ ]:
# We create a training set and a test set (the test set contains 30% of the dataset)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

In [ ]:
# Create arrays and dataframes to store results
oof_preds = np.zeros(df.shape[0])
# sub_preds = np.zeros(test_df.shape[0])

# load model
print("Loading LGBM")
clf_joblib = joblib.load('models/LGBM/LGBMClassifier_fold_0.joblib')

In [ ]:
data.select_dtypes('object') # we don't have 'object' here but it is just in case.


In [ ]:
categorical_columns = df.select_dtypes('object').columns
numerical_columns = df.select_dtypes(include='number').columns
#numerical_columns = X.select_dtypes(include='number').columns

print("Shape of categorical variables : ", categorical_columns.shape)
print("Shape of numerical variables :", numerical_columns.shape)

### Evaluation

In [ ]:
X_test.iloc[:, 1:]

In [ ]:
y_pred_proba = clf_joblib.predict_proba(X_test.iloc[:, 1:])[:, 1] # we remove the ID and we take the predict for the class 1

In [ ]:
y_pred_proba

In [ ]:
def score(estimator):
    """
    Computes and prints train score and test score.
    :param estimator:
    :return:
    """
    tr_score = estimator.score(X_train.iloc[:, 1:], y_train).round(4)
    te_score = estimator.score(X_test.iloc[:, 1:], y_test).round(4)

    print(f"score train : {tr_score} score test : {te_score} ")

In [ ]:
y_pred = clf_joblib.predict(X_test.iloc[:, 1:])#[:, 1] # we remove the ID and we take the predict for the class 1


In [ ]:
y_test

In [ ]:
score(clf_joblib)
confusion(y_test, y_pred)

In [ ]:
f1_score(y_test, y_pred)

In [ ]:
evaluate_classification(y_test, y_pred, y_pred_proba)

In [ ]:
results = evaluate_models("LightGBM", results, y_test, y_pred)

In [ ]:
results

In [ ]:
# End of notebook time
t1 = time()
print("computing time : {:8.6f} sec".format(t1 - t0))
print("computing time : " + strftime('%H:%M:%S', gmtime(t1 - t0)))